In [7]:
from glob import glob
import os
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
imgPath = "../artifacts/data_ingestion/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/data-Splitted"
testPath = imgPath + "/test"
test_dir = os.listdir(testPath)
trainPath = imgPath + "/train"
train_dir = os.listdir(trainPath)
valPath = imgPath + "/val"
val_dir = os.listdir(valPath)
print(testPath, test_dir)
print(trainPath, train_dir)
print(valPath, val_dir)

../artifacts/data_ingestion/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/data-Splitted/test ['Cyst', 'Normal', 'Stone', 'Tumor']
../artifacts/data_ingestion/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/data-Splitted/train ['Cyst', 'Normal', 'Stone', 'Tumor']
../artifacts/data_ingestion/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/data-Splitted/val ['Cyst', 'Normal', 'Stone', 'Tumor']


In [5]:
def imageCount(fPath):
    imgList = [len(glob(fPath + "/" + ele + "/*.jpg")) for ele in os.listdir(fPath)]
    imgSum = sum(imgList)
    return imgSum, imgList

In [6]:
trainCount, trainList = imageCount(trainPath)
print("Total number of images in {} directory : {}".format("TRAIN",trainCount))
valCount, valList = imageCount(valPath)
print("Total number of images in {} directory : {}".format("VALIDATION",valCount))
testCount, testList = imageCount(testPath)
print("Total number of images in {} directory : {}".format("TEST",testCount))

Total number of images in TRAIN directory : 7466
Total number of images in VALIDATION directory : 2487
Total number of images in TEST directory : 2493


## Model building